# Домашнее задание № 5 

В данном задании требуется реализовать некоторые из метрик, рассмотренные на лекции.

Все функции, кроме ```compute_gain```, в качестве первых двух аргументов принимают на вход тензоры ```ys_true``` и ```ys_pred```. Это вещественные тензоры ```pytorch``` размерности ```n```, задающие целевые отметки релевантности и предсказанные значения соответственно. 

Для генерации примеров входных данных можете использовать ```torch.rand(n)```, если не указана специфика исходных тензоров. 

Считается, что ```ys_pred``` содержит уникальные значения без повторений.

In [1]:
import math
from math import log2
import numpy as np

import torch
from torch import Tensor, sort

D:\AnacondaGO\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### I didn't really understand what these metrics mean
https://habr.com/ru/company/econtenta/blog/303458/ - main resource 

## Swapped Pairs

```num_swapped_pairs``` — функция для расчёта количества неправильно упорядоченных пар (корректное упорядочивание — от наибольшего значения в ```ys_true``` к наименьшему) или переставленных пар.

In [2]:
def num_swapped_pairs(ys_true: Tensor, ys_pred: Tensor) -> int:
    mask = torch.argsort(ys_pred, descending = True)
    ys_true_right = ys_true[mask]
    count = 0
    for i in range(len(ys_true_right)):
        count +=sum(ys_true_right[i+1:] > ys_true_right[i])
    return count.item()

In [3]:
# не изменять
ys_true = torch.tensor([2, 1, 0, 1, 2])
ys_pred = torch.tensor([0.1, 0.3, 0.2, 0.13, 0.12])

res = num_swapped_pairs(ys_true, ys_pred)
print(res)  # 7

7


## Gain

```compute_gain``` — вспомогательная функция для расчёта DCG и NDCG, рассчитывающая показатель Gain. Принимает на вход дополнительный аргумент — указание схемы начисления Gain (```gain_scheme```).

Необходимо реализовать метод при:
- ```gain_scheme="const"``` - постоянное начисление Gain
- ```gain_scheme="exp2"``` - рассчитываемый по формуле $(2^r −1)$, где $r$ — реальная релевантность документа некоторому запросу.

In [4]:
def compute_gain(y_value: float, gain_scheme: str) -> float:
    # допишите ваш код здесь
    if gain_scheme == "exp2":
        return pow(2.0, y_value) - 1
    if gain_scheme == "const":
        return y_value
    pass

In [5]:
# не изменять
value = 5

res = compute_gain(value, 'exp2')
print(res)  # 31

res = compute_gain(value, 'const')
print(res)  # 5

31.0
5


## DCG

```dcg``` и ```ndcg``` — функции расчёта DCG и NDCG. Принимают на вход дополнительный параметр ```gain_scheme```, аналогичный таковому в функции ```compute_gain```.

In [6]:
def dcg(ys_true: Tensor, ys_pred: Tensor, gain_scheme: str) -> float:
    mask = torch.argsort(ys_pred, descending = True)
    ys_true_right = ys_true[mask]
    dcg = []
    for idx, val in enumerate(ys_true_right): 
        numerator = compute_gain(val, gain_scheme=gain_scheme)
        # add 2 because python 0-index
        denominator =  np.log2(idx + 2) 
        score = numerator/denominator
        dcg.append(score)
    return sum(dcg).item()


def ndcg(ys_true: Tensor, ys_pred: Tensor, gain_scheme: str = 'const') -> float:
    numerator = dcg(ys_true, ys_pred, gain_scheme)
    mask = torch.argsort(ys_pred, descending = True)
    
    ys_true_right = ys_true[mask]
    ndcg = []
    for idx, val in enumerate(torch.sort(ys_true, descending = True)[0]): 
        num = compute_gain(val, gain_scheme=gain_scheme)
        denominator = np.log2(idx + 2) 
        score = num/denominator
        ndcg.append(score)
    return numerator/sum(ndcg).item()
   

In [7]:
# не изменять
ys_true = torch.tensor([2, 2, 4, 1, 2, 0])
ys_pred = torch.tensor([0.1, 0.3, 0.2, 0.14, 0.12, 0.6])

res = dcg(ys_true, ys_pred, gain_scheme='exp2')
res_n = ndcg(ys_true, ys_pred, gain_scheme='exp2')
print(res)  # 12.052645801815459
print(res_n)  # 0.6004804162123548

12.052645683288574
0.6004803894430065


## Precission@k


```precission_at_k``` — функция расчёта точности в топ-k позиций для бинарной разметки (в ```ys_true``` содержатся только нули и единицы). Если среди лейблов нет ни одного релевантного документа (единицы), то необходимо вернуть -1. 

Функция принимает на вход параметр k, указывающий на то, по какому количеству объектов необходимо произвести расчёт метрики. Учтите, что k может быть больше количества элементов во входных тензорах.

In [8]:
def precission_at_k(ys_true: Tensor, ys_pred: Tensor, k: int) -> float:
    # допишите ваш код здесь
    mask = torch.argsort(ys_pred, descending = True)
    ys_true_right = ys_true[mask]
    if k > len(ys_true_right):
        k = len(ys_true_right)
    if not sum(ys_true_right[:k]):
        return -1
    return sum(ys_true_right[:k]).item()/k

In [9]:
# не изменять
ys_true = torch.tensor([0, 0, 1, 1, 0, 1]) # 0 - не релавнтен, 1 - релевантен, 
ys_pred = torch.tensor([0.1, 0.3, 0.2, 0.14, 0.12, 0.6]) # прдесказанная релевантность

res = precission_at_k(ys_true, ys_pred, k=3) # доля релевантных элементов в предсказании
print(res)  # 0.6666666865348816

0.6666666666666666


## Average Precision

```average_precision``` — функция расчёта AP для бинарной разметки (в ```ys_true``` содержатся только нули и единицы). Если среди лейблов нет ни одного релевантного документа (единицы), то необходимо вернуть -1.

In [10]:
def average_precision(ys_true: Tensor, ys_pred: Tensor) -> float:
    if not sum(ys_true):
        return -1
    
    mask = torch.argsort(ys_pred, descending = True)
    ys_true_right = ys_true[mask]

    res = 0
    for i in range(0, len(ys_true_right)):
        pak = precission_at_k(ys_true, ys_pred, k=i+1)
        res += ys_true_right[i].item()*pak
    return res/sum(ys_true_right>0).item()

In [11]:
# не изменять
ys_true = torch.tensor([1, 0, 1, 1, 0, 1, 0, 0])
ys_pred = torch.tensor([0.9, 0.8, 0.7, 0.6, 0.4, 0.3, 0.2, 0.1])

res = average_precision(ys_true, ys_pred)
print(res)  # 0.7708333333333333

0.7708333333333333


## reciprocal_rank

```reciprocal_rank``` — функция для расчёта MRR (без усреднения). В ```ys_true``` могут содержаться только нули и максимум одна единица. 

In [12]:
def reciprocal_rank(ys_true: Tensor, ys_pred: Tensor) -> float:
    mask = torch.argsort(ys_pred, descending = True)
    ys_true_right = ys_true[mask]
    return 1/(torch.argmax(ys_true_right).item()+1) # +1 because python indexig from 0, maxim one 1, so just argmax

In [13]:
# не изменять
ys_true = torch.tensor([0, 0, 0, 1, 0, 0])
ys_pred = torch.tensor([0.1, 0.3, 0.2, 0.14, 0.12, 0.6])

res = reciprocal_rank(ys_true, ys_pred)
print(res)  # 0.25

0.25


## p_found

```p_found``` — функция расчёта P-found от Яндекса, принимающая на вход дополнительный параметр ```p_break``` — вероятность прекращения просмотра списка документов в выдаче. Базовая вероятность просмотреть первый документ в выдаче ($pLook[0]$) равняется единице. ```ys_true``` нормированы от 0 до 1 (вероятность удовлетворения запроса пользователя).

In [14]:
def p_found(ys_true: Tensor, ys_pred: Tensor, p_break: float = 0.15 ) -> float:
    mask = torch.argsort(ys_pred, descending = True)
    ys_true_right = ys_true[mask]
    
    p_rel = [compute_gain(i, gain_scheme="exp2").item() if i>0 else 0.0 for i in ys_true_right]
    
    p_look = [1]
    for i in range(1, len(p_rel)):
        p_look.append(p_look[i-1] * (1 - p_rel[i-1]) * (1 - p_break))
        
    res = [p_look[i] * p_rel[i] for i in range(len(p_rel))] 
    
    return sum(res)

In [15]:
# не изменять
ys_true = torch.tensor([0, 0, 0, 1, 0, 1])
ys_pred = torch.tensor([0.91, 0.72, 0.12, 0.24, 0.15, 0.6])

res = p_found(ys_true, ys_pred, 0.12)
print(res)  # 0.7744

0.7744
